# Grand Classification Challenge

In [2]:
from sidekit.libsvm import svmutil

Import theano


## Load and Inspect the Data Set

In [ ]:
def svm_read_problem(data_file_name):
    """
    svm_read_problem(data_file_name) -> [y, x]

    Read LIBSVM-format data from data_file_name and return labels y
    and data instances x.
    :param data_file_name: name of the file to load from
    """
    prob_y = []
    prob_x = []
    for line in open(data_file_name):
        line = line.split(None, 1)
        # In case an instance with all zero features
        if len(line) == 1:
            line += ['']
        label, features = line
        xi = {}
        for e in features.split():
            ind, val = e.split(":")
            xi[int(ind)] = float(val)
        prob_y += [float(label)]
        prob_x += [xi]
    return prob_y, prob_x

In [3]:
y_train, X_train = svmutil.svm_read_problem('../data/digits/digit_test')
y_test, X_test = svmutil.svm_read_problem('../data/digits/digit_train')

for(xi in X_train):
    xi

In [10]:
print(type(X_test[0].keys()))

<class 'dict_keys'>
